In [2]:
import os
import numpy as np
import pandas as pd
import pickle
# import sklearn.linear_model as sklm
# import sklearn.model_selection as skms
# import sklearn.feature_selection
import sklearn.pipeline
# from sklearn.utils import shuffle
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neural_network import MLPClassifier
import matplotlib.pyplot as plt

# Import our custom functions
from load_data import load_data 

Load training and validation data:

x data is an array of N*784 pixels (N = 2102 for tr, 600 for va)

y is a dataframe of index, class_name and class_id

In [3]:
x_tr, y_tr_df = load_data('x_train.csv', 'y_train.csv')
x_va, y_va_df = load_data('x_valid.csv', 'y_valid.csv')
x_te = load_data('x_test.csv', 'y_valid.csv')[0]


for label, arr in [('train', x_tr), ('valid', x_va)]:
    print("Contents of %s_x.csv: arr of shape %s" % (
        label, str(arr.shape)))

Contents of train_x.csv: arr of shape (2102, 784)
Contents of valid_x.csv: arr of shape (600, 784)


Plot a random image from the validation data for each category

In [ ]:
# prng = np.random.RandomState(0)
prng = np.random.RandomState()
N = 3 # num examples of each class to show
fig, axgrid = plt.subplots(N, 6, figsize=(6*3, N*2.5))

for ll, label in enumerate(['dress', 'pullover', 'top', 'trouser', 'sandal', 'sneaker']):
    match_df = y_va_df.query("class_name == '%s'" % label)
    match_ids_N = prng.choice(match_df.index, size=N)        
    for ii, row_id in enumerate(match_ids_N):
        ax = axgrid[ii, ll]
        x_SS = x_va[row_id].reshape((28,28))
        ax.imshow(x_SS, vmin=0, vmax=255, cmap='gray')
        ax.set_xticks([]); ax.set_yticks([]);
        if ii == 0:
            ax.set_title(label, fontsize=16)
plt.subplots_adjust(left=0.01, right=0.99, wspace=.2, hspace=.01)
plt.tight_layout();
plt.show();

# Data Exploration 1a

In [4]:
tr_class_dist = y_tr_df['class_name'].value_counts()
val_class_dist = y_va_df['class_name'].value_counts()

print('Training class distribution:')
print(tr_class_dist)
print('Validation class distribution:')
print(val_class_dist)

Training class distribution:
sandal      800
sneaker     800
dress       400
pullover    100
top           1
trouser       1
Name: class_name, dtype: int64
Validation class distribution:
dress       100
trouser     100
sandal      100
top         100
pullover    100
sneaker     100
Name: class_name, dtype: int64


The training data is heavily skewed with data from sandals and trainers, with only one training image for 2 classes. A challenge will be gaining a balanced weighting for each of the classes such that the dominant classes aren't always favoured by the classifier.

# Model

Run search with predefined split s.t. validation set is used for hyperparameter selection

In [5]:
# Combine training and validation datasets
x_all = np.vstack((x_tr,x_va))
y_all_df = pd.concat([y_tr_df,y_va_df])

print("Training X shape: %s\nValidation X shape: %s\nCombined X shape: %s\n" % (x_tr.shape, x_va.shape, x_all.shape))
print("Training Y shape: %s\nValidation Y shape: %s\nCombined Y shape: %s\n" % (y_tr_df.shape, y_va_df.shape, y_all_df.shape))

valid_indicators = np.hstack([
    -1 * np.ones(y_tr_df.shape[0]), # -1 means never include this example in any test split
    0  * np.ones(y_va_df.shape[0]), #  0 means include in the first test split (we count starting at 0 in python)
    ])

# Define custom splitter to use only the validation dataset for hyperparameter selection
print("Splitter dimensions: %i" % (valid_indicators.shape[0]))
my_splitter = sklearn.model_selection.PredefinedSplit(valid_indicators)


Training X shape: (2102, 784)
Validation X shape: (600, 784)
Combined X shape: (2702, 784)

Training Y shape: (2102, 2)
Validation Y shape: (600, 2)
Combined Y shape: (2702, 2)

Splitter dimensions: 2702


In [ ]:
param_dist = dict(activation=['relu', 'logistic', 'identity', 'tanh'], learning_rate_init=np.logspace(-5, 5, 100), learning_rate = ['constant','adaptive'], hidden_layer_sizes=[(20,),(50,),(100,),(200,),(500,)])

fashion_pipes = sklearn.pipeline.Pipeline([
    ('rand_search', RandomizedSearchCV(sklearn.neural_network.MLPClassifier( solver='lbfgs', random_state = 0, shuffle=True, early_stopping = True), param_dist, scoring='balanced_accuracy', error_score='raise', random_state=0, return_train_score=True, n_iter=100, cv= my_splitter, n_jobs = -1, refit=False))
])

fashion_pipes.fit(x_all, y_all_df['class_name'])

In [ ]:
gridRes = fashion_pipes['rand_search'].cv_results_
bestIdx = fashion_pipes['rand_search'].best_index_ # idx 12, 16, 44 all scored the same

bestIdxs = [12,16,44]
bestParams = dict()
bestParams[0] = {k:v[bestIdxs[0]] for k,v in gridRes.items()}
bestParams[1] = {k:v[bestIdxs[1]] for k,v in gridRes.items()}
bestParams[2] = {k:v[bestIdxs[2]] for k,v in gridRes.items()}

display(bestParams)
# print(gridRes["mean_test_score"])
# display(gridRes["mean_test_score"])

In [ ]:
bestMLP1b = sklearn.neural_network.MLPClassifier( solver='lbfgs', random_state = 0, shuffle=True, early_stopping = True, learning_rate_init=1e-05,learning_rate ='constant',hidden_layer_sizes=(500,),activation='identity')
bestMLP1b.fit(x_tr,y_tr_df["class_name"])

In [ ]:
tr_acc = sklearn.metrics.balanced_accuracy_score(y_tr_df['class_name'], bestMLP1b.predict(x_tr))
va_acc = sklearn.metrics.balanced_accuracy_score(y_va_df['class_name'], bestMLP1b.predict(x_va))
print("Training balanced accuracy: %f\nValidation balanced accuracy: %f" % (tr_acc, va_acc))

In [ ]:
pred_tr = bestMLP1b.predict(x_tr)
pred_va = bestMLP1b.predict(x_va)
pred_te = bestMLP1b.predict(x_te)

# Save output of prediction on test data to a file.
np.savetxt('yhat_test.txt', pred_te, delimiter='\n', fmt='%s')

In [ ]:
# Plot test data
rows = 3
cols = 7
fig, axgrid = plt.subplots(rows, cols, layout="constrained")

for imageID in range(rows*cols):
    ax = axgrid[np.unravel_index( imageID, (rows,cols))]
    x_SS = x_te[imageID].reshape((28,28))
    ax.imshow(x_SS, vmin=0, vmax=255, cmap='gray')
    ax.set_xticks([]); ax.set_yticks([]);
    label = pred_te[imageID]
    ax.set_title(label, fontsize=16)
# plt.tight_layout();
fig.suptitle("Sample of test data predictions by model 1B")
plt.show();

In [ ]:
# TODO refine search using gridsearchcv

In [ ]:
# When refit is true we can use the best_estimator_ method, but this doesn't work with refit=false - here we need to manually run a fit on our training set.

# best_est_1 = fashion_pipes['rand_search'].best_estimator_

# pred_tr = best_est_1.predict(x_tr)
# pred_va = best_est_1.predict(x_va)
# pred_te = best_est_1.predict(x_te)

# # Save output of prediction on test data to a file.
# np.savetxt('yhat_test.txt', pred_te, delimiter='\n', fmt='%s')
# sklearn.neural_network.MLPClassifier( solver='lbfgs', random_state = 0, shuffle=True, early_stopping = True)

Ignore for the time being - this was done with refit = true, which fits the final model from randomizedSearchCV onto the entire dataset (tr+val).

![Balanced acc of 1.0 or tr and val](best_estimator_1.png)


# Repeat with data normalized

In [ ]:
# normalize data - necessary
x_tr_norm = x_tr /255
x_va_norm = x_va /255


In [ ]:
# Combine training and validation datasets
x_all_norm = np.vstack((x_tr_norm,x_va_norm))
y_all_df = pd.concat([y_tr_df,y_va_df])

print("Training X shape: %s\nValidation X shape: %s\nCombined X shape: %s\n" % (x_tr_norm.shape, x_va_norm.shape, x_all_norm.shape))
print("Training Y shape: %s\nValidation Y shape: %s\nCombined Y shape: %s\n" % (y_tr_df.shape, y_va_df.shape, y_all_df.shape))

valid_indicators = np.hstack([
    -1 * np.ones(y_tr_df.shape[0]), # -1 means never include this example in any test split
    0  * np.ones(y_va_df.shape[0]), #  0 means include in the first test split (we count starting at 0 in python)
    ])

# Define custom splitter to use only the validation dataset for hyperparameter selection
print("Splitter dimensions: %i" % (valid_indicators.shape[0]))
my_splitter = sklearn.model_selection.PredefinedSplit(valid_indicators)


In [ ]:
param_dist = dict(activation=['relu', 'logistic', 'identity', 'tanh'], learning_rate_init=np.logspace(-5, 5, 100), learning_rate = ['constant','adaptive'], hidden_layer_sizes=[(20,),(50,),(100,),(200,),(500,)])

fashion_pipes_norm = sklearn.pipeline.Pipeline([
    ('rand_search', RandomizedSearchCV(sklearn.neural_network.MLPClassifier( solver='lbfgs', random_state = 0, shuffle=True, early_stopping = True), param_dist, scoring='balanced_accuracy', error_score='raise', random_state=0, return_train_score=True, n_iter=100, cv= my_splitter, n_jobs = -1, refit=False))
])

fashion_pipes_norm.fit(x_all_norm, y_all_df['class_name'])

In [ ]:
# Assess the same procedure with normalized input data - same results and same accuracy found in this case.display

gridRes = fashion_pipes['rand_search'].cv_results_

display(gridRes)
bestIdxNorm = fashion_pipes['rand_search'].best_index_ # idx 12, 16, 44 all scored the same

bestIdxs = [12,16,44]
bestParams = dict()
bestParams[0] = {k:v[bestIdxs[0]] for k,v in gridRes.items()}
bestParams[1] = {k:v[bestIdxs[1]] for k,v in gridRes.items()}
bestParams[2] = {k:v[bestIdxs[2]] for k,v in gridRes.items()}

display(bestParams)
# print(gridRes["mean_test_score"])
# display(gridRes["mean_test_score"])

In [ ]:
bestMLP1bNorm = sklearn.neural_network.MLPClassifier( solver='lbfgs', random_state = 0, shuffle=True, early_stopping = True, learning_rate_init=1e-05,learning_rate ='constant',hidden_layer_sizes=(500,),activation='identity')
bestMLP1bNorm.fit(x_tr_norm,y_tr_df["class_name"])

In [ ]:
tr_acc = sklearn.metrics.balanced_accuracy_score(y_tr_df['class_name'], bestMLP1bNorm.predict(x_tr_norm))
va_acc = sklearn.metrics.balanced_accuracy_score(y_va_df['class_name'], bestMLP1bNorm.predict(x_va_norm))
print("Training balanced accuracy: %f\nValidation balanced accuracy: %f" % (tr_acc, va_acc))

For some reason the validation accuracy seems to be lower on normalized data

In [ ]:
print(best_est_1)

# Model with Duplicate Data

In [6]:
#getting index for each class in training set:
top_int = y_tr_df.index[y_tr_df['class_name']=='top']    #index 1
trous_int = y_tr_df.index[y_tr_df['class_name']=='trouser']    #index 11
dress_int =(y_tr_df.index[y_tr_df['class_name']=='dress']).tolist()
pull_int =(y_tr_df.index[y_tr_df['class_name']=='pullover']).tolist()
sneaker_int =(y_tr_df.index[y_tr_df['class_name']=='sneaker']).tolist()
sandal_int =(y_tr_df.index[y_tr_df['class_name']=='sandal']).tolist()


#sneakers + sandals + tops + trous + dress + pullovers in new dataset.
train_x_2 = np.vstack((x_tr[sandal_int, :], x_tr[sneaker_int, :]))
tops800= np.tile(x_tr[top_int, :], (800, 1)) #make 800 copies of tops
train_x_2 = np.vstack((train_x_2, tops800)) # add 800 tops to sandals + sneakers
assert(train_x_2.shape == (2400, 784))  #checking dimensions

trous800 = np.tile(x_tr[trous_int, :], (800, 1))  #800 copies of trousers
train_x_2 = np.vstack((train_x_2, trous800)) # add 800 trousers
assert(train_x_2.shape == (3200, 784))  #checking dimensions

dress800 = np.tile(x_tr[dress_int, :], (2, 1))   #doubled dresses 400 to 800
assert(dress800.shape == (800, 784))
train_x_2 = np.vstack((train_x_2, dress800)) # add 800 dresses
assert(train_x_2.shape == (4000, 784))  #checking dimensions

pull800 = np.tile(x_tr[pull_int, :], (8, 1))
assert(pull800.shape == (800, 784))
train_x_2 = np.vstack((train_x_2, pull800)) # add 800 dresses
assert(train_x_2.shape == (4800, 784))

#making train_y_2
train_y_2 = np.tile(7, 800) 
train_y_2 = np.hstack((train_y_2, np.tile(5, 800)))
train_y_2 = np.hstack((train_y_2, np.tile(0, 800)))
train_y_2 = np.hstack((train_y_2, np.tile(1, 800)))
train_y_2 = np.hstack((train_y_2, np.tile(3, 800)))
train_y_2 = np.hstack((train_y_2, np.tile(2, 800)))

assert(train_y_2.shape == (4800, ))


In [7]:
# Combine training and validation datasets
x_dup_all = np.vstack((train_x_2, x_va))
y_dup_all = np.hstack((train_y_2, y_va_df['class_uid']))

print("Training X shape: %s\nValidation X shape: %s\nCombined X shape: %s\n" % (train_x_2.shape, x_va.shape, x_dup_all.shape))
print("Training Y shape: %s\nValidation Y shape: %s\nCombined Y shape: %s\n" % (train_y_2.shape, y_va_df['class_uid'].shape, y_dup_all.shape))

valid_dup_indicators = np.hstack([
    -1 * np.ones(train_y_2.shape[0]), #-1 = exclude this example in test split
    0 * np.ones(y_va_df.shape[0]), #0 = include in the first test split
    ])

# Define custom splitter to use only the validation dataset for hyperparameter selection
print("Splitter dimensions: %i" % (valid_dup_indicators.shape[0]))
my_dup_splitter = sklearn.model_selection.PredefinedSplit(valid_dup_indicators)

Training X shape: (4800, 784)
Validation X shape: (600, 784)
Combined X shape: (5400, 784)

Training Y shape: (4800,)
Validation Y shape: (600,)
Combined Y shape: (5400,)

Splitter dimensions: 5400


In [ ]:
param_dist = dict(activation=['relu', 'logistic', 'identity', 'tanh'], learning_rate_init=np.logspace(-5, 5, 100), learning_rate = ['constant','adaptive'], hidden_layer_sizes=[(20,),(50,),(100,),(200,),(500,)], random_state = [0, 1, 2, 3, 4, 5, 6])

#### With just 4 iterations in Randomized Search CV

In [ ]:
loaded_model4 = pickle.load(open('1D_4_iter_initial_search.sav', 'rb'))
result4 = loaded_model4.score(x_dup_all, y_dup_all)
print(result4)

Both of the below parameters with n_iter = 4 had same test score 0.445

In [ ]:
{'random_state': 3,
 'learning_rate_init': 4.037017258596558e-05,
 'learning_rate': 'constant',
 'hidden_layer_sizes': (100,),
 'activation': 'logistic'}

{'random_state': 6,
   'learning_rate_init': 4.037017258596558e-05,
   'learning_rate': 'constant',
   'hidden_layer_sizes': (500,),
   'activation': 'identity'}

#### With just 16 iterations in Randomized Search CV

In [ ]:
param_dist16 = dict(learning_rate_init=np.logspace(-5, 5, 100), learning_rate = ['constant','adaptive'], hidden_layer_sizes=[(20,),(50,),(100,),(200,),(500,)], random_state = [0, 1, 2, 3, 4, 5, 6])

In [ ]:
loaded_model16 = pickle.load(open('1D_16_iter_initial_search.sav', 'rb'))
result16 = loaded_model16.score(x_dup_all, y_dup_all)
print(result16)

In [ ]:
{'random_state': 5,
   'learning_rate_init': 954.5484566618328,
   'learning_rate': 'adaptive',
   'hidden_layer_sizes': (20,),
   'activation': 'tanh'},
   
{'random_state': 4,
   'learning_rate_init': 0.00041320124001153346,
   'learning_rate': 'constant',
   'hidden_layer_sizes': (20,),
   'activation': 'identity'},

### 64 iterations Model

In [ ]:
loaded_model64 = pickle.load(open('1D_64_iter_initial_search.sav', 'rb'))
result64 = loaded_model64.score(x_dup_all, y_dup_all)
print(result4)

In [38]:
dup_fashion_pipes64['rand_search'].cv_results_ 
#Best score was 0.486. Same as when we did 16 iters. Not necessary to do 64
{'random_state': 4,
   'learning_rate_init': 0.00041320124001153346,
   'learning_rate': 'constant',
   'hidden_layer_sizes': (20,),
   'activation': 'identity'}
{'random_state': 4,
   'learning_rate_init': 0.021544346900318846,
   'learning_rate': 'adaptive',
   'hidden_layer_sizes': (20,),
   'activation': 'identity'}

{'mean_fit_time': array([ 12.98047519,  57.53519821,  61.01048613, 101.27840877,
        154.14830899,  14.68329239,  35.75348592,  54.83108926,
        123.53866339,  14.62633729,   4.29979658,   4.4848721 ,
         19.99948883,  20.77584076,  61.2773695 ,  96.31079388]),
 'std_fit_time': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'mean_score_time': array([0.02224731, 0.00521946, 0.0167346 , 0.01674795, 0.02640319,
        0.01655626, 0.01300073, 0.01639867, 0.01231408, 0.01347518,
        0.        , 0.        , 0.01666713, 0.00688744, 0.01781321,
        0.03361917]),
 'std_score_time': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'param_random_state': masked_array(data=[2, 5, 3, 6, 5, 5, 6, 6, 2, 5, 4, 4, 2, 5, 3, 0],
              mask=[False, False, False, False, False, False, False, False,
                    False, False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 '

### Now narrowing - 16 iters 20 layers

In [49]:
flag1 = 'go'

#Gave activation function logistic and hidden layer 20. Gave 200 iterations.
#   ALL came out to have scores 0.528333... Trying random seeds 0-4

if flag1 != 'stop':
    param_dist2 = dict(learning_rate_init=np.logspace(-5, 5, 100), learning_rate = ['constant','adaptive'], random_state=[0, 1, 2, 3, 4])

    dup_fashion_pipes2 = sklearn.pipeline.Pipeline([
        ('rand_search', RandomizedSearchCV(sklearn.neural_network.MLPClassifier (activation='identity', solver='lbfgs', shuffle=True, early_stopping = True, hidden_layer_sizes=(20,)), param_dist2, scoring='balanced_accuracy', error_score='raise', return_train_score=True, n_iter=16, cv= my_dup_splitter, n_jobs = -1, refit= False))
    ])

    dup_fashion_pipes2.fit(x_dup_all, y_dup_all)
    # filename = '1D_second_RanSearch.sav'
    # pickle.dump(dup_fashion_pipes, open(filename, 'wb'))

In [50]:
scores2 = dup_fashion_pipes2['rand_search'].cv_results_['mean_test_score']
print(scores2)
#best scores on index 10 and 13 with 0.54!

[0.48666667 0.37333333 0.54166667 0.37333333 0.30333333 0.30333333
 0.37333333 0.54166667 0.48666667 0.37333333 0.48666667 0.48666667
 0.48333333 0.30333333 0.48333333 0.30333333]


In [53]:
dup_fashion_pipes2['rand_search'].best_params_

{'random_state': 1,
 'learning_rate_init': 0.5590810182512223,
 'learning_rate': 'adaptive'}

In [ ]:
{'random_state': 1,
   'learning_rate_init': 0.5590810182512223,
   'learning_rate': 'adaptive'},
{'random_state': 1,
   'learning_rate_init': 24770.76355991714,
   'learning_rate': 'constant'},

Let's try grid search

In [ ]:

param_dist3 = dict(learning_rate_init=np.logspace(-5, 0, 2), learning_rate = ['constant','adaptive'], random_state=[0, 1, 2, 3, 4])

dup_fashion_pipes3 = sklearn.pipeline.Pipeline([
    ('grid_search', sklearn.GridSearchCV(sklearn.neural_network.MLPClassifier (activation='identity', solver='lbfgs', shuffle=True, early_stopping = True, hidden_layer_sizes=(20,)), param_dist2, scoring='balanced_accuracy', error_score='raise', return_train_score=True, n_iter=16, cv= my_dup_splitter, n_jobs = -1, refit= False))
])

dup_fashion_pipes3.fit(x_dup_all, y_dup_all)
# filename = '1D_second_RanSearch.sav'
# pickle.dump(dup_fashion_pipes, open(filename, 'wb'))

c:\Users\leigh\micromamba\envs\cs135_env\lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
c:\Users\leigh\micromamba\envs\cs135_env\lib\site-packages\sklearn\metrics\_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
c:\Users\leigh\micromamba\envs\cs135_env\lib\site-packages\sklearn\metrics\_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
c:\Users\leigh\micromamba\envs\cs135_env\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize

Pipeline(steps=[('rand_search',
                 RandomizedSearchCV(cv=5, error_score='raise',
                                    estimator=MLPClassifier(random_state=0,
                                                            solver='lbfgs'),
                                    n_iter=5,
                                    param_distributions={'activation': ['relu',
                                                                        'logistic',
                                                                        'identity',
                                                                        'tanh'],
                                                         'batch_size': [10, 25,
                                                                        50, 100,
                                                                        200,
                                                                        500],
                                                         'learning_rate_init': array([1.00000000e-05, 1.26185688e-05, 1.59228279e-05, 2.00923300e-05,
       2.53536449e-05,...
       1.20450354e+03, 1.51991108e+03, 1.91791026e+03, 2.42012826e+03,
       3.05385551e+03, 3.85352859e+03, 4.86260158e+03, 6.13590727e+03,
       7.74263683e+03, 9.77009957e+03, 1.23284674e+04, 1.55567614e+04,
       1.96304065e+04, 2.47707636e+04, 3.12571585e+04, 3.94420606e+04,
       4.97702356e+04, 6.28029144e+04, 7.92482898e+04, 1.00000000e+05])},
                                    random_state=0, return_train_score=True,
                                    scoring='balanced_accuracy'))])